### Titanic: Machine Learning from Disaster:
### Objective: Predict survival on the Titanic

In [ ]:
# Please find below a summary of the overall approach.
# 1. Chi-square test was performed to sigle out of the feature importance of the catagorical feature and it was concluded that All catagorical features seems to have some lind of influence on the outcome.
# 2.Following Features have been decided to be removed from model building and training as part of initial phase.
#         -- Cabin : because most of the values (Aprox 70%) are missing 
#         -- Names:
#         -- Ticket
# 3.Model Accuracy has been tested with Cross validations score using following models.
#       1.Decesion Trees Classifier
#       2.RandomForest Classifier
#       3.LogisticRegression
#       4.Support Vector Classifier
#        5.GradientBoosting Classifier
#        6.XGBoost Classifier
#        7.XGBoost Random Forest Classifier

# Based on 10 Crosss validation splits, it is found that the XGBoost Random Forest Classifier gave the better accuracy score of Min Score of:75%, Maximum Score of:90% and Average Score of: 83%

# 4. K-Fold Crosss validation was performed and extracted the best training and Testing sample out of 10 splits and model was retrained.
# Belwo scores were observed:
# **Train Score :87% and  Test Score:90%  **

In [1]:
import numpy as np
import pandas as pd

In [2]:
dataSet=pd.read_csv(r'D:\SarojOfficeWork\AI\Kaggle\TitanicSurviver\train.csv')

In [5]:
dataSet.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [6]:
# Check the dataset for features and Label types
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
# Check for Null Features and % of Null values across each Featur
# Features having Null values are 'Age','Cabin' and 'Embarked'
dataSet.isnull().sum()/len(dataSet)*100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [3]:
# Fill in Null Values for 'Age' with mean value
dataSet.Age.fillna(np.round(dataSet.Age.mean()),inplace=True)
# Fill in Null Values for 'Embarked' with mode
dataSet.Embarked.fillna(dataSet.Embarked.mode()[0],inplace=True)

# Since there are 77% of missing values for the attribute Cabin, its better to discard this columns from model building as part of initial trial
# Lets also drop PassengerId,Name column from model building
dataSet1=dataSet.drop(columns=['PassengerId','Name','Cabin'])
dataSet1.head()
# Name           891 non-null object
# Embarked       889 non-null object
# Cabin          204 non-null object
# Sex            891 non-null object
# Ticket         891 non-null object

Survived  Pclass     Sex   Age  SibSp  Parch            Ticket     Fare  \
0         0       3    male  22.0      1      0         A/5 21171   7.2500   
1         1       1  female  38.0      1      0          PC 17599  71.2833   
2         1       3  female  26.0      0      0  STON/O2. 3101282   7.9250   
3         1       1  female  35.0      1      0            113803  53.1000   
4         0       3    male  35.0      0      0            373450   8.0500   

  Embarked  
0        S  
1        C  
2        S  
3        S  
4        S

In [97]:
# Lets Make Age into Age_group  (Optional Step)
dataSet1.Age.describe()

count    891.000000
mean      29.758889
std       13.002570
min        0.420000
25%       22.000000
50%       30.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [100]:
def age_group(x):
    age_grp=''
    if (x >=0 and x <= 12):
        age_grp='Kid'
    elif (x >=13 and x <= 19):
        age_grp='Teenager'
    elif (x >=20 and x <= 55):
        age_grp='Adult'
    else:
        age_grp='Older'
    return  age_grp   

In [101]:
dataSet1['Age_group']=dataSet1.Age.apply(age_group)

In [102]:
dataSet1

Survived  Pclass     Sex   Age  SibSp  Parch            Ticket     Fare  \
0           0       3    male  22.0      1      0         A/5 21171   7.2500   
1           1       1  female  38.0      1      0          PC 17599  71.2833   
2           1       3  female  26.0      0      0  STON/O2. 3101282   7.9250   
3           1       1  female  35.0      1      0            113803  53.1000   
4           0       3    male  35.0      0      0            373450   8.0500   
..        ...     ...     ...   ...    ...    ...               ...      ...   
886         0       2    male  27.0      0      0            211536  13.0000   
887         1       1  female  19.0      0      0            112053  30.0000   
888         0       3  female  30.0      1      2        W./C. 6607  23.4500   
889         1       1    male  26.0      0      0            111369  30.0000   
890         0       3    male  32.0      0      0            370376   7.7500   

    Embarked Age_group  
0          S     Adult  
1          C     Adult  
2          S     Adult  
3          S     Adult  
4          S     Adult  
..       ...       ...  
886        S     Adult  
887        S  Teenager  
888        S     Adult  
889        C     Adult  
890        Q     Adult  

[891 rows x 10 columns]

In [135]:
# Lets try to figure out: Among all categorical features which features seems to be influncing most for predicting the outcome
# We will do Chisquare test for SELECTING TOP-N CATEGORICAL FEATURES. For this lets separate all categorical features and label
dataSet2=dataSet1.drop(columns=['Age','Fare','SibSp','Parch'])
dataSet2=dataSet2.astype(str)
dataSet2.head()

#From Below Chisquare Test it seems most influencing Categorical feature is 'Ticket' followed by 'Sex','Pclass'.
#However the scores are not that far apart and hence its not conclusive to leave out any acategorical feature at this point.
# We can possibly leave out Ticket Ticket and include Fare instead
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import OrdinalEncoder
features=dataSet2.drop(columns=['Survived'])
label=dataSet2.loc[:,['Survived']]
OE=OrdinalEncoder()
features_chi=OE.fit_transform(features)  # This is needed to encode each catagorical variable to integers values.
modelKBest=SelectKBest(score_func=chi2,k='all')
finalFeatures=modelKBest.fit_transform(features_chi,label)
print(features.columns)
print(modelKBest.scores_)

Index(['Pclass', 'Sex', 'Ticket', 'Embarked', 'Age_group'], dtype='object')
[5.44658660e+01 9.27024470e+01 2.87165547e+03 1.02025247e+01
 2.02415975e+00]


In [64]:
# Lets decide Final features and  label as below. Dropping out PassengerId,Name,Cabin, Ticket columns for the initial model. 
finalFeatures=dataSet.drop(columns=['PassengerId','Name','Cabin','Ticket','Survived'])
label=dataSet.loc[:,['Survived']]

In [65]:
# Lets apply OneHotEncoding for Categorical features
finalFeatures=pd.get_dummies(finalFeatures)
finalFeatures.head()

Pclass   Age  SibSp  Parch     Fare  Sex_female  Sex_male  Embarked_C  \
0       3  22.0      1      0   7.2500           0         1           0   
1       1  38.0      1      0  71.2833           1         0           1   
2       3  26.0      0      0   7.9250           1         0           0   
3       1  35.0      1      0  53.1000           1         0           0   
4       3  35.0      0      0   8.0500           0         1           0   

   Embarked_Q  Embarked_S  
0           0           1  
1           0           0  
2           0           1  
3           0           1  
4           0           1

In [66]:
finalFeatures=finalFeatures.values
label=label.values

In [71]:
import warnings
warnings.filterwarnings('ignore')

# Lets apply Cross validation score to know how much of accuracy we can expect from this data set.
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBRFClassifier,XGBClassifier
from sklearn.svm import SVC
cv=10

modelDT=DecisionTreeClassifier()
modelRFC=RandomForestClassifier()
modelLR=LogisticRegression()
modelGBC=GradientBoostingClassifier()
modelXGBRFC=XGBRFClassifier()
modelXGBC=XGBClassifier()
modelSVC=SVC()

cvDT=cross_val_score(cv=cv,estimator=modelDT,X=finalFeatures,y=label)
print('Crosss Validation scores for Decesion Trees are',np.round(cvDT,2))

cvRFC=cross_val_score(cv=cv,estimator=modelRFC,X=finalFeatures,y=label)
print('Crosss Validation scores for RandomForest Classifier are',np.round(cvRFC,2))

cvLR=cross_val_score(cv=cv,estimator=modelLR,X=finalFeatures,y=label)
print('Crosss Validation scores for LogisticRegression are',np.round(cvLR,2))

cvSVC=cross_val_score(cv=cv,estimator=modelSVC,X=finalFeatures,y=label)
print('Crosss Validation scores for Support Vector Classifier are',np.round(cvSVC,2))

cvGBC=cross_val_score(cv=cv,estimator=modelGBC,X=finalFeatures,y=label)
print('Crosss Validation scores for GradientBoostingClassifier are',np.round(cvGBC,2))

cvXGBC=cross_val_score(cv=cv,estimator=modelXGBC,X=finalFeatures,y=label)
print('Crosss Validation scores for XGBoostClassifier are',np.round(cvXGBC,2))

cvXGBRFC=cross_val_score(cv=cv,estimator=modelXGBRFC,X=finalFeatures,y=label)
print('Crosss Validation scores for XGBoostRandomForestClassi are Min Score:{}, Maximum Score:{}, Average Score:{}'.format(
    round(np.min(cvXGBRFC),2),round(np.max(cvXGBRFC),2),round(np.average(cvXGBRFC),2)
    )
     )

Crosss Validation scores for Decesion Trees are [0.72 0.8  0.73 0.8  0.82 0.76 0.83 0.72 0.81 0.82]
Crosss Validation scores for RandomForest Classifier are [0.77 0.78 0.74 0.87 0.88 0.82 0.81 0.78 0.79 0.84]
Crosss Validation scores for LogisticRegression are [0.78 0.79 0.78 0.85 0.79 0.78 0.79 0.78 0.81 0.83]
Crosss Validation scores for Support Vector Classifier are [0.67 0.63 0.73 0.73 0.73 0.74 0.76 0.75 0.75 0.82]
Crosss Validation scores for GradientBoostingClassifier are [0.8  0.81 0.75 0.88 0.88 0.85 0.84 0.79 0.84 0.83]
Crosss Validation scores for XGBoostClassifier are [0.78 0.82 0.75 0.83 0.85 0.83 0.83 0.79 0.87 0.82]
Crosss Validation scores for XGBoostRandomForestClassi are Min Score:0.75, Maximum Score:0.9, Average Score:0.83


### Conclusion sofar
#XGBoostRandomForestClassifier is expected to give better scores than others.
#Min Score:0.75, Maximum Score:0.9, Average Score:0.83

In [68]:
# Now lets try to get the Scores using K-Fold Cross Validation

#Initialize the algo
model=XGBRFClassifier()

#Initialize KFold Method
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, #n_splits should be equal to no of cv value in cross_val_score
              random_state=1,
              shuffle=True)

#Initialize For Loop 

i=0
for train,test in kfold.split(finalFeatures):
    i = i+1
    X_train,X_test = features[train],features[test]
    y_train,y_test = label[train],label[test]
    
    model.fit(X_train,y_train)
    
    #if model.score(X_test,y_test) >= 0.95:
    print("Train Score: {}, Test score: {}, for Sample Split: {}".format(model.score(X_train,y_train),model.score(X_test,y_test),i))
        

Train Score: 0.8701622971285893, Test score: 0.7888888888888889, for Sample Split: 1
Train Score: 0.8753117206982544, Test score: 0.797752808988764, for Sample Split: 2
Train Score: 0.8790523690773068, Test score: 0.7752808988764045, for Sample Split: 3
Train Score: 0.8753117206982544, Test score: 0.8202247191011236, for Sample Split: 4
Train Score: 0.8703241895261845, Test score: 0.7865168539325843, for Sample Split: 5
Train Score: 0.8690773067331671, Test score: 0.8651685393258427, for Sample Split: 6
Train Score: 0.85785536159601, Test score: 0.8539325842696629, for Sample Split: 7
Train Score: 0.871571072319202, Test score: 0.8089887640449438, for Sample Split: 8
Train Score: 0.8665835411471322, Test score: 0.898876404494382, for Sample Split: 9
Train Score: 0.871571072319202, Test score: 0.8089887640449438, for Sample Split: 10


In [72]:
#Lets extract the Train and Test sample for split 9
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, #n_splits should be equal to no of cv value in cross_val_score
              random_state=1,
              shuffle=True)
i=0
for train,test in kfold.split(finalFeatures):
    i = i+1
    if i == 9:
        X_train,X_test,y_train,y_test = finalFeatures[train],finalFeatures[test],label[train],label[test]

In [145]:
#Final Model
finalModel=XGBRFClassifier()
finalModel.fit(X_train,y_train)
train_score=np.round(finalModel.score(X_train,y_train),2)
test_score=np.round(finalModel.score(X_test,y_test),2)
print('XGBRFClassifier Train Score is:{} and  Test Score:{}'.format(train_score,test_score))

XGBRFClassifier Train Score is:0.87 and  Test Score:0.9


In [159]:
# Confusion Matrix and Classification Report
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(y_true=label, y_pred=finalModel.predict(finalFeatures))
CR=classification_report(y_true=label, y_pred=finalModel.predict(finalFeatures))
print('Confusion Matrix:\n',cm)
print('\n Classification Report:\n',CR)

Confusion Matrix:
 [[513  36]
 [ 80 262]]

 Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.93      0.90       549
           1       0.88      0.77      0.82       342

    accuracy                           0.87       891
   macro avg       0.87      0.85      0.86       891
weighted avg       0.87      0.87      0.87       891



### Making the Test data ready for Predictions

In [81]:
testData=pd.read_csv(r'D:\SarojOfficeWork\AI\Kaggle\TitanicSurviver\test.csv')
testData.isnull().sum()/len(dataSet)*100

PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age             9.652076
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.112233
Cabin          36.700337
Embarked        0.000000
dtype: float64

In [85]:
testData.head(1)

PassengerId  Pclass              Name   Sex   Age  SibSp  Parch  Ticket  \
0          892       3  Kelly, Mr. James  male  34.5      0      0  330911   

     Fare Cabin Embarked  
0  7.8292   NaN        Q

In [115]:
# Fill in Null Values for 'Age' with mean value
testData.Age.fillna(np.round(testData.Age.mean()),inplace=True)

# Fill in Null Values for 'Fare' with mean value
testData.Fare.fillna(np.round(testData.Fare.mean()),inplace=True)

# Lets make the Final features same as Train dataset
finalTestdata=testData.drop(columns=['PassengerId','Name','Cabin','Ticket'])

# Lets apply OneHotEncoding for Categorical features
finalTestdata=pd.get_dummies(finalTestdata).values

# Preddict the survivir based on test data
survivor=finalModel.predict(finalTestdata)

# Export the prediction as CSV file
prediction=testData[['PassengerId']]
prediction['Survived']=pd.DataFrame(survivor,columns=['Survived'])
prediction.to_csv('D:\SarojOfficeWork\AI\Kaggle\TitanicSurviver\prediction.csv')